In [1]:
import torch
import torch.nn as nn
import numpy
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
rate=0.001
epoch=20
batch_size = 32

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

print(device)

cpu


In [4]:
mnist_train = dsets.MNIST( root = 'MNIST_data/',
                          train= True,
                          transform = transforms.ToTensor(),
                          download=True)
 
mnist_test = dsets.MNIST( root = 'MNIST_data/',
                          train= False,
                          transform = transforms.ToTensor(),
                          download=True)

In [5]:
data = torch.utils.data.DataLoader(mnist_train, batch_size, shuffle=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32, kernel_size = 3 , stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size = 3 , stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size = 3 , stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc=nn.Linear(3*3*128,10, bias=True)
        self.fc1 = nn.Linear(3*3*128,625)
        self.fc2 = nn.Linear(625,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:
model = CNN().to(device)
cr = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = rate)

In [8]:
total_batch = len(data)

for e in range(epoch):
    avg_cost = 0
    for x,y in data:
        optimizer.zero_grad()
        x=x.to(device)
        y=y.to(device)
        
        hy=model(x)
        cost = cr(hy, y)
        
        cost.backward()
        optimizer.step()
        avg_cost +=cost / total_batch
    if e%5==0:
        print('[Epoch: {}] cost = {}'.format(e, avg_cost))


print("학습 완료")

[Epoch: 0] cost = 0.13945406675338745
[Epoch: 5] cost = 0.020449241623282433
[Epoch: 10] cost = 0.013781735673546791
[Epoch: 15] cost = 0.013035654090344906
학습 완료


In [9]:
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    pr = model(x_test)
    co_pr = torch.argmax(pr, 1) == y_test
    accuracy = co_pr.float().mean()
    print("Accuracy:",accuracy.item()*100)

print("테스트 완료")

C:\Users\kuroc\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\kuroc\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 94.4599986076355
테스트 완료
